In [0]:
#  /FileStore/tables/sales_csv.txt
#  /FileStore/tables/menu_csv.txt

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
sales_schema=StructType([StructField("item_id",IntegerType()),
                         StructField("customer_id",StringType()),
                         StructField("order_date",DateType()),
                         StructField("country",StringType()),
                         StructField("order_type",StringType())])

In [0]:
menu_schema=StructType([StructField("item_id",IntegerType()),
                        StructField("item_name",StringType()),
                        StructField("item_price",StringType())])

In [0]:
sales_df=spark.read.format("csv").schema(sales_schema).option("inferSchema","true").load("/FileStore/tables/sales_csv.txt")
sales_df.show()

+-------+-----------+----------+-------+----------+
|item_id|customer_id|order_date|country|order_type|
+-------+-----------+----------+-------+----------+
|      1|          A|2023-01-01|  India|    Swiggy|
|      2|          A|2022-01-01|  India|    Swiggy|
|      2|          A|2023-01-07|  India|    Swiggy|
|      3|          A|2023-01-10|  India|Restaurant|
|      3|          A|2022-01-11|  India|    Swiggy|
|      3|          A|2023-01-11|  India|Restaurant|
|      2|          B|2022-02-01|  India|    Swiggy|
|      2|          B|2023-01-02|  India|    Swiggy|
|      1|          B|2023-01-04|  India|Restaurant|
|      1|          B|2023-02-11|  India|    Swiggy|
|      3|          B|2023-01-16|  India|    zomato|
|      3|          B|2022-02-01|  India|    zomato|
|      3|          C|2023-01-01|  India|    zomato|
|      1|          C|2023-01-01|     UK|    Swiggy|
|      6|          C|2022-01-07|     UK|    zomato|
|      3|          D|2023-02-16|     UK|Restaurant|
|      5|   

In [0]:
sales_df.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- country: string (nullable = true)
 |-- order_type: string (nullable = true)



In [0]:
menu_df = spark.read.option("delimiter", ",").schema(menu_schema).csv("/FileStore/tables/menu_csv.txt")
menu_df.show()
menu_df.printSchema()

+-------+---------+----------+
|item_id|item_name|item_price|
+-------+---------+----------+
|      1|    PIZZA|       100|
|      2|  Chowmin|       150|
|      3| sandwich|       120|
|      4|     Dosa|       110|
|      5|  Biryani|        80|
|      6|    Pasta|       180|
+-------+---------+----------+

root
 |-- item_id: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- item_price: string (nullable = true)



In [0]:
menu_df=menu_df.withColumn("item_price",col("item_price").cast(DecimalType(5,1)))
menu_df.show()
menu_df.printSchema()

+-------+---------+----------+
|item_id|item_name|item_price|
+-------+---------+----------+
|      1|    PIZZA|     100.0|
|      2|  Chowmin|     150.0|
|      3| sandwich|     120.0|
|      4|     Dosa|     110.0|
|      5|  Biryani|      80.0|
|      6|    Pasta|     180.0|
+-------+---------+----------+

root
 |-- item_id: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- item_price: decimal(5,1) (nullable = true)



In [0]:
sales_df=sales_df.withColumnRenamed("item_id","dish_id")
sales_menu_df=sales_df.join(menu_df,sales_df["dish_id"]==menu_df["item_id"],"inner")
sales_menu_df.show()

+-------+-----------+----------+-------+----------+-------+---------+----------+
|dish_id|customer_id|order_date|country|order_type|item_id|item_name|item_price|
+-------+-----------+----------+-------+----------+-------+---------+----------+
|      1|          A|2023-01-01|  India|    Swiggy|      1|    PIZZA|     100.0|
|      2|          A|2022-01-01|  India|    Swiggy|      2|  Chowmin|     150.0|
|      2|          A|2023-01-07|  India|    Swiggy|      2|  Chowmin|     150.0|
|      3|          A|2023-01-10|  India|Restaurant|      3| sandwich|     120.0|
|      3|          A|2022-01-11|  India|    Swiggy|      3| sandwich|     120.0|
|      3|          A|2023-01-11|  India|Restaurant|      3| sandwich|     120.0|
|      2|          B|2022-02-01|  India|    Swiggy|      2|  Chowmin|     150.0|
|      2|          B|2023-01-02|  India|    Swiggy|      2|  Chowmin|     150.0|
|      1|          B|2023-01-04|  India|Restaurant|      1|    PIZZA|     100.0|
|      1|          B|2023-02

In [0]:
display(sales_menu_df.groupBy(col("customer_id")).agg(sum(col("item_price")).alias("revenue")).orderBy(col("revenue").desc()))

customer_id,revenue
B,4440.0
A,4260.0
C,2400.0
E,2040.0
D,1200.0


Databricks visualization. Run in Databricks to view.

In [0]:
display(sales_menu_df.groupBy(col("item_name")).agg(sum(col("item_price")).alias("revenue")).orderBy(col("revenue").desc()))

item_name,revenue
sandwich,5760.0
Chowmin,3600.0
PIZZA,2100.0
Dosa,1320.0
Pasta,1080.0
Biryani,480.0


Databricks visualization. Run in Databricks to view.

In [0]:
display(sales_menu_df.groupBy(month(col("order_date")).alias("order_month")).agg(sum(col("item_price")).alias("revenue")).orderBy(col("revenue").desc()))

order_month,revenue
1,2960.0
6,2960.0
5,2960.0
2,2730.0
3,910.0
7,910.0
11,910.0


Databricks visualization. Run in Databricks to view.

In [0]:
display(sales_menu_df.groupBy(quarter(col("order_date")).alias("order_quarter")).agg(sum(col("item_price")).alias("revenue")).orderBy(col("revenue").desc()))

order_quarter,revenue
1,6600.0
2,5920.0
3,910.0
4,910.0


Databricks visualization. Run in Databricks to view.

In [0]:
display(sales_menu_df.groupBy(year(col("order_date")).alias("order_year")).agg(sum(col("item_price")).alias("revenue")).orderBy(col("revenue").desc()))

order_year,revenue
2023,9990.0
2022,4350.0


Databricks visualization. Run in Databricks to view.

In [0]:
display(sales_menu_df.groupBy(col("item_name")).agg(count(col("item_id")).alias("total_orders")).orderBy(col("total_orders").desc()))

item_name,total_orders
sandwich,48
Chowmin,24
PIZZA,21
Dosa,12
Pasta,6
Biryani,6


Databricks visualization. Run in Databricks to view.

In [0]:
display(sales_menu_df.groupBy(col("item_name")).agg(count(col("item_id")).alias("total_orders")).orderBy(col("total_orders").desc()).limit(5))

item_name,total_orders
sandwich,48
Chowmin,24
PIZZA,21
Dosa,12
Pasta,6


Databricks visualization. Run in Databricks to view.

In [0]:
display(sales_menu_df.groupBy(col("item_name")).agg(count(col("item_id")).alias("total_orders")).orderBy(col("total_orders").desc()).limit(1))

item_name,total_orders
sandwich,48


Databricks visualization. Run in Databricks to view.

In [0]:
display(sales_menu_df.groupBy(col("customer_id")).agg(count(col("item_id")).alias("total_visits")).orderBy(col("total_visits").desc()))

customer_id,total_visits
B,36
A,33
E,18
C,18
D,12


Databricks visualization. Run in Databricks to view.

In [0]:
display(sales_menu_df.groupBy(col("country")).agg(count(col("item_id")).alias("total_orders")).orderBy(col("total_orders").desc()))

country,total_orders
UK,57
India,39
USA,21


Databricks visualization. Run in Databricks to view.

In [0]:
display(sales_menu_df.groupBy(col("order_type")).agg(count(col("item_id")).alias("total_orders")).orderBy(col("total_orders").desc()))

order_type,total_orders
Swiggy,51
zomato,39
Restaurant,27


Databricks visualization. Run in Databricks to view.